# PyTorch Linear Regression with custom data generatig process with added noise

## Problem statement

We want to predict the crop yield of *apples* and *oranges* based on the 3 input variables *temperature*, *rainfall* and *humidity*.
Due to measurement errors we have to deal with an unobservable noise component:
* apples = 0.5\*temperature + 0.5\*rainfall + 0.2\*humidity + nc
* oranges = 1\*temperature + 0.2\*rainfall - 0.1\*humidity + nc

In [1]:
# Import PyTorch, NumPy
import numpy as np
import torch
from torch.nn import functional as F
import torch.nn as nn

## Data generating process

In [24]:
# Define input variables
temperature = torch.randint(20, 70, size=(100,), requires_grad=False)
rainfall = torch.randint(0, 100, size=(100,), requires_grad=False)
humidity = torch.randint(0, 100, size=(100,), requires_grad=False)
inputs = torch.stack([temperature, rainfall, humidity], dim=1).type(torch.float32)
print(inputs[:4,:])

tensor([[22., 89., 61.],
        [66., 77., 20.],
        [31., 21., 43.],
        [44.,  6., 86.]])


In [50]:
# Calculate target variables based on DGP (with noise component)
torch.manual_seed(0)  # fix seed for reproducability
apples_coeff = torch.tensor([0.5, 0.5, 0.2])
oranges_coeff = torch.tensor([1, 0.2, -0.1])
nc = torch.randn(size=(100,)) * 2  # scale by 2 to make it a little larger, since torch.randn has mean 0 and std 1
apples = torch.sum(inputs*apples_coeff, dim=1) + nc
oranges = torch.sum(inputs*oranges_coeff, dim=1) + nc
targets = torch.stack([apples, oranges], dim=1).type(torch.float32)

In [51]:
targets[:5,]

tensor([[65.4483, 31.4483],
        [73.1953, 77.0953],
        [34.0988, 30.3988],
        [41.3322, 35.7322],
        [51.1974, 46.5974]])

## Dataset, DataLoader

In [32]:
from torch.utils.data import TensorDataset, DataLoader

In [52]:
ds_train = TensorDataset(inputs, targets)

In [54]:
batch_size = 10
dl_train = DataLoader(dataset=ds_train, batch_size=batch_size, shuffle=True)

## SimpleNet

In [56]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(3, 10)
        self.act1 = nn.ReLU()
        self.linear2 = nn.Linear(10, 2)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.act1(x)
        x = self.linear2(x)
        return x

In [57]:
# Instantiate model
model = SimpleNet()